In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import gc
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Display all columns 
pd.pandas.set_option('display.max_columns', None)

In [3]:
train_eval = pd.read_csv("/content/drive/MyDrive/Colab/sales_train_evaluation.csv")
calendar = pd.read_csv("/content/drive/MyDrive/Colab/calendar.csv")
sell_prices = pd.read_csv("/content/drive/MyDrive/Colab/sell_prices.csv")
sample = pd.read_csv("/content/drive/MyDrive/Colab/sample_submission.csv")

In [ ]:
# View Data Information
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
train_eval.head()

id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  d_23  d_24  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  d_35  d_36  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  d_47  d_48  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     2     0     0     0     2     0     1     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_49  d_50  d_51  d_52  d_53  d_54  d_55  d_56  d_57  d_58  d_59  d_60  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     2     0     1     0     0     1     1     1     0     2     3   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_61  d_62  d_63  d_64  d_65  d_66  d_67  d_68  d_69  d_70  d_71  d_72  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     1     0     0     0     0     0     1     0     0     1     1     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_73  d_74  d_75  d_76  d_77  d_78  d_79  d_80  d_81  d_82  d_83  d_84  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     1     0     0     1     2     3     0     2     0     0     2   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_85  d_86  d_87  d_88  d_89  d_90  d_91  d_92  d_93  d_94  d_95  d_96  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   


In [ ]:
# view data shape
train_eval.shape,calendar.shape,sell_prices.shape

((30490, 1947), (1969, 14), (6841121, 4))

In [ ]:
# check the null values
train_eval.columns[train_eval.isnull().sum() > 0]

Index([], dtype='object')

In [ ]:
calendar.columns[calendar.isnull().sum() > 0]

Index(['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], dtype='object')

In [ ]:
calendar.isnull().sum().sort_values(ascending = False)

event_type_2    1964
event_name_2    1964
event_type_1    1807
event_name_1    1807
snap_WI            0
snap_TX            0
snap_CA            0
d                  0
year               0
month              0
wday               0
weekday            0
wm_yr_wk           0
date               0
dtype: int64

In [4]:
# add Zero sales for dates d_1942 to d_1969
for i in range(1942,1970):
    col = "d_"+ str(i)
    train_eval[col] = 0

In [5]:
train_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1975 entries, id to d_1969
dtypes: int64(1969), object(6)
memory usage: 459.4+ MB


In [6]:
# Memory Usage Reduction
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [7]:
# Downcast in order to save memory
train_eval = downcast(train_eval)
sell_prices = downcast(sell_prices)
calendar = downcast(calendar)

In [8]:
# check memory usage after downcast
train_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1975 entries, id to d_1969
dtypes: category(6), int16(1317), int8(652)
memory usage: 97.4 MB


In [ ]:
train_eval.head()

id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  d_23  d_24  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  d_35  d_36  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  d_47  d_48  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     2     0     0     0     2     0     1     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_49  d_50  d_51  d_52  d_53  d_54  d_55  d_56  d_57  d_58  d_59  d_60  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     2     0     1     0     0     1     1     1     0     2     3   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_61  d_62  d_63  d_64  d_65  d_66  d_67  d_68  d_69  d_70  d_71  d_72  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     1     0     0     0     0     0     1     0     0     1     1     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_73  d_74  d_75  d_76  d_77  d_78  d_79  d_80  d_81  d_82  d_83  d_84  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     1     0     0     1     2     3     0     2     0     0     2   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_85  d_86  d_87  d_88  d_89  d_90  d_91  d_92  d_93  d_94  d_95  d_96  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   


In [9]:
# MELT the Dataset
melt_df = pd.melt(train_eval, 
          id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
          var_name = 'd', 
          value_name = "sales")
melt_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [ ]:
melt_df[melt_df['sales']>0].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2
14,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4
15,HOBBIES_1_016_CA_1_evaluation,HOBBIES_1_016,HOBBIES_1,HOBBIES,CA_1,CA,d_1,5
21,HOBBIES_1_022_CA_1_evaluation,HOBBIES_1_022,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2


In [10]:
# Merge the datasets
merge_df = pd.merge(melt_df,calendar, on = "d")
merge_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


In [11]:
# memoryClear
del calendar,melt_df,train_eval
gc.collect()

145

In [12]:
merge_df = pd.merge(merge_df, sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left') 
merge_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [13]:
del sell_prices
gc.collect()

132

In [14]:
# Buliding LGBM model
 # convert numeric variables into categorical variables

conv_id = dict(zip(merge_df.id.cat.codes, merge_df.id))                
conv_cat_id = dict(zip(merge_df.cat_id.cat.codes, merge_df.cat_id))


In [15]:
merge_df.d = merge_df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = merge_df.dtypes.index.tolist()
types = merge_df.dtypes.values.tolist()

In [16]:
merge_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [17]:
for i,type in enumerate(types):
    if type.name == 'category':
        merge_df[cols[i]] = merge_df[cols[i]].cat.codes

In [ ]:
merge_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,14370,1437,3,1,0,0,1,0,2011-01-29,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
1,14380,1438,3,1,0,0,1,0,2011-01-29,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
2,14390,1439,3,1,0,0,1,0,2011-01-29,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
3,14400,1440,3,1,0,0,1,0,2011-01-29,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN
4,14410,1441,3,1,0,0,1,0,2011-01-29,11101,2,1,1,2011,-1,-1,-1,-1,0,0,0,NaN


In [18]:
merge_df.drop('date',1,inplace = True)

In [ ]:
# LGBM Model Training 

#create  dataframes
valid = merge_df[(merge_df['d']>=1914) & (merge_df['d']<1942)][['id','d','sales']]
test = merge_df[merge_df['d']>=1942][['id','d','sales']]
eval_preds = test['sales']
valid_preds = valid['sales']

    

In [ ]:
eval_preds_all=[]
valid_preds_all=[]
cats = merge_df.cat_id.astype('category').cat.codes.unique().tolist()
max_depth_group=[6,4]

In [19]:

for i in range(2):
  #create  dataframes
  valid = merge_df[(merge_df['d']>=1914) & (merge_df['d']<1942)][['id','d','sales']]
  test = merge_df[merge_df['d']>=1942][['id','d','sales']]
  eval_preds = test['sales']
  valid_preds = valid['sales']
  max_depth_choose=max_depth_group[i]
  print('  **** Prediction for max_depth: {}****  '.format(max_depth_group[i]))
  for cat in cats:
    df = merge_df[merge_df['cat_id']==cat]
    
    # split the data into train,validate and test
    x_train, y_train = df[df['d']<1914].drop('sales',axis=1), df[df['d']<1914]['sales']
    x_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sales',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sales']
    x_test = df[df['d']>=1942].drop('sales',axis=1)
    
    #model
    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=max_depth_choose,
        num_leaves=50,
        min_child_weight=300
    )
    print('##Prediction for Category: {}##'.format(conv_cat_id[cat]))
    model.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_valid,y_valid)],
             eval_metric='rmse', verbose=20, early_stopping_rounds=10)
    valid_preds[x_valid.index] = model.predict(x_valid)
    eval_preds[x_test.index] = model.predict(x_test)
    del model, x_train, y_train, x_valid, y_valid, df
    gc.collect()
  valid_preds_all.append(valid_preds)
  eval_preds_all.append(eval_preds)
  del valid,test,eval_preds,valid_preds

  **** Prediction for max_depth: 6****  
##Prediction for Category: HOBBIES##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 2.97487	training's rmse: 1.72478	valid_1's l2: 2.93413	valid_1's rmse: 1.71293
[40]	training's l2: 2.89169	training's rmse: 1.7005	valid_1's l2: 2.86194	valid_1's rmse: 1.69173
[60]	training's l2: 2.83483	training's rmse: 1.68369	valid_1's l2: 2.82659	valid_1's rmse: 1.68125
[80]	training's l2: 2.80789	training's rmse: 1.67568	valid_1's l2: 2.81673	valid_1's rmse: 1.67831
[100]	training's l2: 2.78467	training's rmse: 1.66873	valid_1's l2: 2.7981	valid_1's rmse: 1.67275
[120]	training's l2: 2.76793	training's rmse: 1.66371	valid_1's l2: 2.78799	valid_1's rmse: 1.66973
[140]	training's l2: 2.75407	training's rmse: 1.65954	valid_1's l2: 2.77375	valid_1's rmse: 1.66546
[160]	training's l2: 2.7436	training's rmse: 1.65638	valid_1's l2: 2.76681	valid_1's rmse: 1.66337
[180]	training's l2: 2.73238	training's rmse: 1.65299	valid_1's l2:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##Prediction for Category: FOODS##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 17.3774	training's rmse: 4.16862	valid_1's l2: 15.2677	valid_1's rmse: 3.9074
[40]	training's l2: 16.038	training's rmse: 4.00475	valid_1's l2: 13.9625	valid_1's rmse: 3.73664
[60]	training's l2: 15.3327	training's rmse: 3.9157	valid_1's l2: 13.3367	valid_1's rmse: 3.65194
[80]	training's l2: 14.5825	training's rmse: 3.81871	valid_1's l2: 12.6858	valid_1's rmse: 3.56171
[100]	training's l2: 14.1961	training's rmse: 3.76777	valid_1's l2: 12.3604	valid_1's rmse: 3.51573
[120]	training's l2: 13.8279	training's rmse: 3.71858	valid_1's l2: 12.0536	valid_1's rmse: 3.47183
[140]	training's l2: 13.2381	training's rmse: 3.63842	valid_1's l2: 11.6867	valid_1's rmse: 3.41858
[160]	training's l2: 13.0017	training's rmse: 3.60579	valid_1's l2: 11.3944	valid_1's rmse: 3.37555
[180]	training's l2: 12.7819	training's rmse: 3.57517	valid_1's l2: 11.2108	valid_1's rmse: 3.34825
[200]	trai

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  **** Prediction for max_depth: 4****  
##Prediction for Category: HOBBIES##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 3.19008	training's rmse: 1.78608	valid_1's l2: 3.20206	valid_1's rmse: 1.78943
[40]	training's l2: 3.10692	training's rmse: 1.76265	valid_1's l2: 3.11271	valid_1's rmse: 1.76429
[60]	training's l2: 3.05917	training's rmse: 1.74905	valid_1's l2: 3.02366	valid_1's rmse: 1.73887
[80]	training's l2: 3.02863	training's rmse: 1.74029	valid_1's l2: 2.99644	valid_1's rmse: 1.73102
[100]	training's l2: 2.99499	training's rmse: 1.7306	valid_1's l2: 2.95302	valid_1's rmse: 1.71844
[120]	training's l2: 2.97181	training's rmse: 1.72389	valid_1's l2: 2.94234	valid_1's rmse: 1.71532
[140]	training's l2: 2.95641	training's rmse: 1.71942	valid_1's l2: 2.92705	valid_1's rmse: 1.71086
[160]	training's l2: 2.94001	training's rmse: 1.71465	valid_1's l2: 2.91091	valid_1's rmse: 1.70614
[180]	training's l2: 2.91987	training's rmse: 1.70876	valid_1's l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##Prediction for Category: FOODS##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 20.7843	training's rmse: 4.55898	valid_1's l2: 18.3765	valid_1's rmse: 4.28679
[40]	training's l2: 19.6329	training's rmse: 4.4309	valid_1's l2: 17.2057	valid_1's rmse: 4.14798
[60]	training's l2: 18.6194	training's rmse: 4.31502	valid_1's l2: 16.5573	valid_1's rmse: 4.06907
[80]	training's l2: 18.0291	training's rmse: 4.24607	valid_1's l2: 15.8729	valid_1's rmse: 3.98407
[100]	training's l2: 17.7063	training's rmse: 4.20789	valid_1's l2: 15.5365	valid_1's rmse: 3.94164
[120]	training's l2: 17.2093	training's rmse: 4.14841	valid_1's l2: 15.0714	valid_1's rmse: 3.88219
[140]	training's l2: 16.7721	training's rmse: 4.09538	valid_1's l2: 14.672	valid_1's rmse: 3.8304
[160]	training's l2: 16.3317	training's rmse: 4.04124	valid_1's l2: 14.1839	valid_1's rmse: 3.76615
[180]	training's l2: 16.0101	training's rmse: 4.00126	valid_1's l2: 13.8894	valid_1's rmse: 3.72685
[200]	trai

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


IndexError: ignored

In [24]:
valid = merge_df[(merge_df['d']>=1914) & (merge_df['d']<1942)][['id','d','sales']]
test = merge_df[merge_df['d']>=1942][['id','d','sales']]
eval_preds = test['sales']
valid_preds = valid['sales']
print('  **** Prediction for max_depth: {}****  '.format(3))
for cat in cats:
    df = merge_df[merge_df['cat_id']==cat]
    
    # split the data into train,validate and test
    x_train, y_train = df[df['d']<1914].drop('sales',axis=1), df[df['d']<1914]['sales']
    x_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sales',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sales']
    x_test = df[df['d']>=1942].drop('sales',axis=1)
    
    #model
    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        max_depth=3,
        num_leaves=50,
        min_child_weight=300
    )
    print('##Prediction for Category: {}##'.format(conv_cat_id[cat]))
    model.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_valid,y_valid)],
             eval_metric='rmse', verbose=20, early_stopping_rounds=10)
    valid_preds[x_valid.index] = model.predict(x_valid)
    eval_preds[x_test.index] = model.predict(x_test)
    del model, x_train, y_train, x_valid, y_valid, df
    gc.collect()
valid_preds_all.append(valid_preds)
eval_preds_all.append(eval_preds)

  **** Prediction for max_depth: 3****  
##Prediction for Category: HOBBIES##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 3.27173	training's rmse: 1.80879	valid_1's l2: 3.27716	valid_1's rmse: 1.81029
[40]	training's l2: 3.21439	training's rmse: 1.79287	valid_1's l2: 3.19605	valid_1's rmse: 1.78775
[60]	training's l2: 3.16338	training's rmse: 1.77859	valid_1's l2: 3.14529	valid_1's rmse: 1.7735
[80]	training's l2: 3.13927	training's rmse: 1.7718	valid_1's l2: 3.1189	valid_1's rmse: 1.76604
[100]	training's l2: 3.11727	training's rmse: 1.76558	valid_1's l2: 3.09792	valid_1's rmse: 1.76009
[120]	training's l2: 3.09672	training's rmse: 1.75975	valid_1's l2: 3.06645	valid_1's rmse: 1.75113
[140]	training's l2: 3.07912	training's rmse: 1.75474	valid_1's l2: 3.04825	valid_1's rmse: 1.74592
[160]	training's l2: 3.06122	training's rmse: 1.74964	valid_1's l2: 3.04334	valid_1's rmse: 1.74452
[180]	training's l2: 3.05148	training's rmse: 1.74685	valid_1's l2:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##Prediction for Category: FOODS##
Training until validation scores don't improve for 10 rounds.
[20]	training's l2: 21.8945	training's rmse: 4.67916	valid_1's l2: 19.0401	valid_1's rmse: 4.3635
[40]	training's l2: 21.0982	training's rmse: 4.59328	valid_1's l2: 18.4901	valid_1's rmse: 4.30002
[60]	training's l2: 20.6481	training's rmse: 4.54401	valid_1's l2: 18.1695	valid_1's rmse: 4.26257
[80]	training's l2: 19.9793	training's rmse: 4.46982	valid_1's l2: 17.6607	valid_1's rmse: 4.20247
[100]	training's l2: 19.4685	training's rmse: 4.41231	valid_1's l2: 17.1075	valid_1's rmse: 4.13612
[120]	training's l2: 19.051	training's rmse: 4.36475	valid_1's l2: 16.8057	valid_1's rmse: 4.09948
[140]	training's l2: 18.7317	training's rmse: 4.32801	valid_1's l2: 16.4326	valid_1's rmse: 4.05371
[160]	training's l2: 18.5558	training's rmse: 4.30764	valid_1's l2: 16.2466	valid_1's rmse: 4.0307
[180]	training's l2: 18.2878	training's rmse: 4.27642	valid_1's l2: 16.0719	valid_1's rmse: 4.00898
[200]	trai

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
## best iteration is for max_depth: 6****   (Minimum of rmse)
valid_preds_best=valid_preds_all[0]
eval_preds_best=eval_preds_all[0]

In [27]:
# Final Submission
valid['sales'] = valid_preds_best
validation = valid[['id','d','sales']]
validation = pd.pivot(validation, index='id', columns='d', values='sales').reset_index()
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation.id = validation.id.map(conv_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sales'] = eval_preds_best
evaluation = test[['id','d','sales']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sales').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(conv_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit.to_csv('submission.csv',index=False)